In [29]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape
import pickle
import json
from sklearn.externals.joblib import Memory
from census import Census
import requests

memory = Memory(location='/tmp', verbose=0)
pd.options.display.max_columns = 999

In [8]:
##### LOAD LICENSE DATA FOR THE FIRST TIME
# make sure these column aren't numerical
lcs_dtype = {
    'LICENSE ID': str,
    'ACCOUNT NUMBER': str,
    'WARD': str,
    'PRECINCT': str,
    'POLICE DISTRICT': str,
    'LICENSE CODE': str,
    'LICENSE NUMBER': str,
    'SSA': str
}

# date columns to parse
lcs_dates = [
    'APPLICATION CREATED DATE', 
    'APPLICATION REQUIREMENTS COMPLETE', 
    'PAYMENT DATE',
    'LICENSE TERM START DATE',
    'LICENSE TERM EXPIRATION DATE',
    'LICENSE APPROVED FOR ISSUANCE',
    'DATE ISSUED',
    'LICENSE STATUS CHANGE DATE'
]
# load from csv
lcs = pd.read_csv('data/Business_Licenses.csv', dtype=lcs_dtype, parse_dates=lcs_dates)

In [10]:
def gdf_from_latlong(df, lat, long):
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[long], df[lat]))
    return gdf

In [11]:
lcs = gdf_from_latlong(lcs, lat='LATITUDE', long='LONGITUDE')

In [12]:
# save licenses gdf for later 
pickle.dump(lcs, open("pickle/lcs", "wb" ))

In [13]:
##### LOAD LICENSE DATA SUBSEQEUNTLY
lcs = pickle.load(open("pickle/lcs", "rb" ))

In [17]:
lcs.dtypes

ID                                           object
LICENSE ID                                   object
ACCOUNT NUMBER                               object
SITE NUMBER                                   int64
LEGAL NAME                                   object
DOING BUSINESS AS NAME                       object
ADDRESS                                      object
CITY                                         object
STATE                                        object
ZIP CODE                                     object
WARD                                         object
PRECINCT                                     object
WARD PRECINCT                                object
POLICE DISTRICT                              object
LICENSE CODE                                 object
LICENSE DESCRIPTION                          object
BUSINESS ACTIVITY ID                         object
BUSINESS ACTIVITY                            object
LICENSE NUMBER                               object
APPLICATION 

In [14]:
lcs.head()

,ID,LICENSE ID,ACCOUNT NUMBER,SITE NUMBER,LEGAL NAME,DOING BUSINESS AS NAME,ADDRESS,CITY,STATE,ZIP CODE,WARD,PRECINCT,WARD PRECINCT,POLICE DISTRICT,LICENSE CODE,LICENSE DESCRIPTION,BUSINESS ACTIVITY ID,BUSINESS ACTIVITY,LICENSE NUMBER,APPLICATION TYPE,APPLICATION CREATED DATE,APPLICATION REQUIREMENTS COMPLETE,PAYMENT DATE,CONDITIONAL APPROVAL,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED,LICENSE STATUS,LICENSE STATUS CHANGE DATE,SSA,LATITUDE,LONGITUDE,LOCATION,geometry
0,22308-20060816,1723393,29481,1,BELL OIL TERMINAL INC,Bell Oil Terminal LLC,3741 S PULASKI RD 1,CHICAGO,IL,60623,14,NaN,14-,8,1010,Limited Business License,NaN,NaN,22308,RENEW,NaT,2006-06-21,2006-08-10,N,2006-08-16,2007-08-15,2006-08-10,2006-08-11,AAI,NaT,NaN,41.825320,-87.723960,"(41.82531992987547, -87.72395999659746)",POINT (-87.72395999700001 41.82531993)
1,1620668-20160516,2455262,295026,1,BUCCI BIG & TALL INC.,BUCCI BIG & TALL INC.,558 W ROOSEVELT RD,CHICAGO,IL,60607,25,28,25-28,1,1010,Limited Business License,911,Retail Sales of Clothing / Accessories / Shoes,1620668,RENEW,NaT,2016-03-15,2016-05-18,N,2016-05-16,2018-05-15,2016-05-18,2016-08-30,AAI,NaT,NaN,41.867339,-87.641590,"(41.86733856638269, -87.64159005699716)",POINT (-87.641590057 41.867338566)
2,2368602-20160616,2460909,291461,3,"PROJECT: VISION , INC.","PROJECT : VISION , INC",2301 S ARCHER AVE 1 1,CHICAGO,IL,60616,25,18,25-18,9,1625,Raffles,720,Not-For-Profit Selling Raffles for Prizes of $...,2368602,RENEW,NaT,2016-04-15,2016-06-21,N,2016-06-16,2017-06-15,2016-06-21,2016-06-22,AAC,2016-08-30,NaN,41.850843,-87.638734,"(41.85084294374687, -87.63873424399071)",POINT (-87.63873424399999 41.850842944)
3,2060891-20141016,2353257,357247,1,FOLASHADE'S CLEANING SERVICE INC.,FOLASHADE'S CLEANING SERVICE INC.,1965 BERNICE RD 1 1SW,LANSING,IL,60438,NaN,NaN,NaN,NaN,1010,Limited Business License,NaN,NaN,2060891,RENEW,NaT,2014-08-15,2016-04-01,N,2014-10-16,2016-10-15,2016-04-01,2016-04-01,AAI,NaT,38,41.951316,-87.678586,"(41.95131555606832, -87.67858578019546)",POINT (-87.67858578000001 41.951315556)
4,1144216-20070516,1804790,147,63,WALGREEN CO.,Walgreens # 05192,9148 S COMMERCIAL AVE 1ST,CHICAGO,IL,60617,10,25,10-25,4,1010,Limited Business License,NaN,NaN,1144216,RENEW,NaT,2007-03-23,2007-05-10,N,2007-05-16,2008-05-15,2007-05-10,2007-05-11,AAI,NaT,5,41.728622,-87.551366,"(41.72862173556932, -87.55136646594693)",POINT (-87.551366466 41.728621736)


In [ ]:
# get acs 5-year estimate for each year (block group, cook county)

# current problems:
# 1) earliest 2009, latest 2017 -- doesn't cover all of our periods
# 2) block group level only available after 2013

In [67]:
# ACS variables
# potentially also work-residence distance?

ACS_VAR_DIC = {
    'NAME': 'NAME',
    'B01003_001E': 'population_total',
    'B02001_002E': 'population_white',
    'B02001_003E': 'population_black',
    'B02001_004E': 'population_american_indian_and_alaskan_native',
    'B02001_005E': 'population_asian',
    'B02001_006E': 'population_native_hawaiian',
    'B02001_007E': 'population_some_other_race_alone',
    'B02001_008E': 'population_two_or_more_races',
    'B02001_009E': 'population_two_races_including_some_other_race',
    'B02001_010E': 'population_two_races_excluding_some_other_race_and_three_or_more_races',
    'B23006_002E': 'education_less_than_high_school',
    'B23006_009E': 'education_high_school',
    'B23006_016E': 'education_some_college_or_associate',
    'B23006_023E': 'education_bachelors_or_higher',
    'B19013_001E': 'median_income'
}
ACS_VAR_LIST = tuple([v for v in var_dic.keys()])
ACS_GEO_DIC = {'for': 'block group:*', 
               'in': 'state:17 county:031'}
ACS_API_KEY = '68c71cf3327ee04e568b590529287b186460bdd4'

In [68]:
def get_acs_data(minyear, maxyear, api_key, var_list, geo_dic, output_directory):
    '''
    get acs5 data for a year range and store them in the output directory
    '''
    c = Census(api_key)
    for year in range(minyear, maxyear+1):
        print("obtaining data from {}...".format(year))
        result = c.acs5.get(var_list, geo_dic, year=year)
        # write to json file
        output_filepath = output_directory + "acs5_{}_{}.json".format(
                          year, geo_dic['for'][:-2])
        with open(output_filepath, 'w') as f:
            json.dump(test, f)
    print('finished')

In [69]:
# get acs5 est for all block groups in cook county 2013-2017
get_acs_data(2013, 2017, ACS_API_KEY, ACS_VAR_LIST, ACS_GEO_DIC, 'data/')

obtaining data from 2013...
obtaining data from 2014...
obtaining data from 2015...
obtaining data from 2016...
obtaining data from 2017...
finished


In [64]:
# load ACS data
def load_json(filepath):
        return json.load(open(filepath, 'r'))